# 📊 MOEX ISS API — Учебное пособие

Добро пожаловать! В этом notebook вы научитесь:

1. Подключаться к API Московской Биржи
2. Загружать исторические данные по индексам
3. Анализировать и визуализировать данные

**Необходимые библиотеки:**
- `pandas` — работа с таблицами
- `requests` — HTTP запросы
- `matplotlib` — визуализация (опционально)

## 📚 Часть 1: Что такое MOEX ISS API?

**MOEX ISS** (Information & Statistical Server) — это открытый API Московской Биржи.

**Что можно получить:**
- Котировки акций, облигаций, валют
- Значения индексов (IMOEX, RGBI и др.)
- Справочную информацию об инструментах

**Официальная документация:** https://iss.moex.com/iss/reference/

## 🔧 Часть 2: Установка и импорт

In [ ]:
# Установка пакета (выполните один раз)
# !pip install -e ..

# Или установите зависимости напрямую:
# !pip install pandas requests

In [ ]:
# Импорт библиотек
import sys
sys.path.insert(0, '..')  # Добавляем путь к пакету

from moex_iss import MOEXClient
from moex_iss.indices import EQUITY_INDICES, BOND_INDICES, list_indices

import pandas as pd

# Опционально: для визуализации
try:
    import matplotlib.pyplot as plt
    plt.style.use('seaborn-v0_8-whitegrid')
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    print("matplotlib не установлен. Графики будут недоступны.")

print("✅ Библиотеки импортированы!")

## 🚀 Часть 3: Первые шаги

In [ ]:
# Создаём клиент API
client = MOEXClient()

print("Клиент MOEX ISS создан!")
print(f"Базовый URL: {client.BASE_URL}")

In [ ]:
# Посмотрим структуру API — какие "движки" доступны
engines = client.get_engines()
engines[['name', 'title']]

In [ ]:
# Рынки в рамках фондового движка (stock)
markets = client.get_markets('stock')
markets.head(10)

## 📈 Часть 4: Загрузка данных по индексам

In [ ]:
# Получим список всех доступных индексов
all_indices = client.get_available_indices()

print(f"Всего индексов на MOEX: {len(all_indices)}")
all_indices[['SECID', 'SHORTNAME']].head(15)

In [ ]:
# Загружаем данные по индексу IMOEX (основной индекс Московской биржи)
imoex = client.get_index_history('IMOEX', start_date='2024-01-01')

print(f"Загружено {len(imoex)} записей")
imoex.head()

In [ ]:
# Базовая статистика
print("📊 Статистика индекса IMOEX:")
print(f"   Период: {imoex['TRADEDATE'].min().strftime('%Y-%m-%d')} — {imoex['TRADEDATE'].max().strftime('%Y-%m-%d')}")
print(f"   Минимум: {imoex['CLOSE'].min():.2f}")
print(f"   Максимум: {imoex['CLOSE'].max():.2f}")
print(f"   Среднее: {imoex['CLOSE'].mean():.2f}")
print(f"   Стд. отклонение: {imoex['CLOSE'].std():.2f}")

## 📉 Часть 5: Визуализация (если установлен matplotlib)

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Строим график динамики IMOEX
    fig, ax = plt.subplots(figsize=(12, 5))
    
    ax.plot(imoex['TRADEDATE'], imoex['CLOSE'], linewidth=1.5, color='#1f77b4')
    ax.fill_between(imoex['TRADEDATE'], imoex['CLOSE'], alpha=0.3)
    
    ax.set_title('Динамика индекса IMOEX', fontsize=14, fontweight='bold')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Значение индекса')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Для построения графиков установите matplotlib: pip install matplotlib")

## 📊 Часть 6: Сравнение нескольких индексов

In [ ]:
# Загрузим несколько секторных индексов
indices_to_compare = {
    'IMOEX': 'Индекс МосБиржи',
    'MOEXOG': 'Нефть и газ',
    'MOEXFN': 'Финансы',
    'MOEXIT': 'IT сектор'
}

data = {}
for code, name in indices_to_compare.items():
    print(f"Загружаем {name} ({code})...")
    df = client.get_index_history(code, start_date='2024-01-01')
    if not df.empty:
        data[code] = df
        print(f"   ✅ Загружено {len(df)} записей")
    else:
        print(f"   ❌ Данные не найдены")

print("\nГотово!")

In [ ]:
# Нормализуем данные для сравнения (приводим к 100 на начало периода)
normalized = pd.DataFrame()

for code, df in data.items():
    # Берём первое значение как базу
    base_value = df['CLOSE'].iloc[0]
    normalized[code] = (df.set_index('TRADEDATE')['CLOSE'] / base_value) * 100

normalized.head()

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Строим сравнительный график
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for i, (code, name) in enumerate(indices_to_compare.items()):
        if code in normalized.columns:
            ax.plot(normalized.index, normalized[code], 
                   label=f'{name} ({code})', 
                   linewidth=2, 
                   color=colors[i])
    
    ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
    
    ax.set_title('Сравнение секторных индексов MOEX (нормализовано к 100)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Индекс (начало = 100)')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 📋 Часть 7: Справочник индексов

In [ ]:
# Просмотр всех индексов акций из нашего справочника
print("📈 ИНДЕКСЫ АКЦИЙ\n" + "="*50)
for code, info in EQUITY_INDICES.items():
    print(f"{code:15} │ {info['name_ru']}")

In [ ]:
# Просмотр основных индексов облигаций
print("📊 ИНДЕКСЫ ОБЛИГАЦИЙ (основные)\n" + "="*50)
for code, info in list(BOND_INDICES.items())[:15]:
    print(f"{code:15} │ {info['name_ru']}")
print(f"\n... и ещё {len(BOND_INDICES) - 15} индексов облигаций")

In [ ]:
# Получить полную таблицу всех индексов
all_available = list_indices('all')
print(f"Всего в справочнике: {len(all_available)} индексов")
all_available.head(20)

## 💾 Часть 8: Сохранение данных

In [ ]:
# Сохраняем данные в CSV
output_file = 'imoex_data.csv'

success = client.download_to_csv('IMOEX', output_file, start_date='2024-01-01')

if success:
    print(f"✅ Данные сохранены в {output_file}")
else:
    print("❌ Ошибка сохранения")

In [ ]:
# Или сохраняем напрямую через pandas
imoex.to_csv('imoex_full.csv', index=False)
print("✅ Сохранено через pandas!")

## 🎯 Упражнения для самостоятельной работы

1. **Загрузите данные по индексу RGBITR** (государственные облигации) за 2023-2024 год и постройте график

2. **Сравните IT-сектор (MOEXIT) с широким рынком (IMOEX)** — какой индекс показал лучшую динамику?

3. **Рассчитайте дневную доходность** для IMOEX: `return = (close_t / close_{t-1}) - 1`

4. **Найдите дни с максимальным падением** индекса IMOEX

In [ ]:
# Место для вашего кода
# ...


## 📚 Полезные ссылки

- [MOEX ISS API Reference](https://iss.moex.com/iss/reference/)
- [Руководство разработчика MOEX ISS (PDF)](https://fs.moex.com/files/8888)
- [Список индексов MOEX](https://www.moex.com/ru/indices)
- [Методология расчёта индексов](https://www.moex.com/s2532)